# Pandas и большие файлы

In [1]:
import pandas as pd

### Упражнение
Для каждого пользователя user_id из файла sales_db.csv посчитайте самую дорогую покупку (в столбце cost)

In [3]:
data = pd.read_csv('sales_db.csv')
data

,user_id,category,cost
0,1,Электроника,87
1,1,Дача,10
2,1,Дети,2
3,1,Дача,199
4,2,Авто,3
5,2,Авто,35
6,3,Дети,8
7,9,Дети,17


In [4]:
data.groupby('user_id').max().head()

,category,cost
user_id,,
1,Электроника,199
2,Авто,35
3,Дети,8
9,Дети,17


In [5]:
data.groupby('user_id').max()['cost'].reset_index().sort_values(by='cost', ascending = False).head()

,user_id,cost
0,1,199
1,2,35
3,9,17
2,3,8


In [12]:
previous_user = None
total_sum = []

with open('sales_db.csv') as f:
    f.readline()
    
    for line in f:
        line = line.strip().split(',')

        current_user_id = line[0]
        cost = int(line[2])
#         print(current_user_id, cost)
        
        if previous_user:
            # сравнение с current_user_id
            if previous_user == current_user_id:
                total_sum.append(cost)
            else:
                print(previous_user, max(total_sum))
                
                previous_user = current_user_id
                total_sum = [cost]
        else:
            previous_user = current_user_id
            total_sum.append(cost)
            
print(previous_user, max(total_sum))

1 199
2 35
3 8
9 17


# Объединение датафреймов

### Данные со слайдов

In [ ]:
visits = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 77],
        'source': ['ad', 'yandex', 'email', 'google', 'ad']
    }
)

visits = visits[['user_id', 'source']]
visits

In [ ]:
purchases = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'category': ['Спорт', 'Авто', 'Дача', 'Спорт', 'Авто'],
    }
)

purchases = purchases[['user_id', 'category']]
purchases

In [ ]:
visits_grouped = visits.groupby('user_id').count()
visits_grouped.rename(columns={'source': 'visits'}, inplace=True)
visits_grouped

In [ ]:
visits.groupby('user_id').count().reset_index()

In [ ]:
purchases_pivot = purchases.pivot_table(index='user_id', columns='category', values='user_id', 
                                        aggfunc='size', fill_value=0)
purchases_pivot

In [ ]:
purchases_pivot.reset_index()

In [ ]:
visits_grouped.join(purchases_pivot)

### LEFT join
Каждой строчке в левой таблице ищет соответствие в правой

In [ ]:
visits_grouped.join(purchases_pivot, how='left')

### RIGHT join
Каждой строчке в правой таблице ищет соответствие в левой

In [ ]:
visits_grouped.join(purchases_pivot, how='right')

### Упражнение
Дана статистика:
- ID клиентов и их имена (датафрейм clients)
- статистика доходов (earnings)
- статистика расходов (spending)

Определите имена клиентов, расходы которых превышают доходы.

In [19]:
# подсказка - по умолчанию в методе merge объединение НЕ left join

?pd.DataFrame.merge

In [13]:
clients = pd.DataFrame(
    {
        'id': [43018, 48329, 51043, 74943, 75029],
        'name': ['Марков Илья', 'Зарицкая Елизавета', 'Благова Дарья', 'Слепова Елена', 'Гордецкий Максим'],
    }
)

clients

,id,name
0,43018,Марков Илья
1,48329,Зарицкая Елизавета
2,51043,Благова Дарья
3,74943,Слепова Елена
4,75029,Гордецкий Максим


In [14]:
earnings = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'debit': [34500, 12400, 89044, 5355, 19800],
    }
)

earnings

,id,debit
0,51043,34500
1,48329,12400
2,74943,89044
3,75029,5355
4,43018,19800


In [15]:
spending = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'credit': [22990, 2500, 69880, 6000, 29000],
    }
)

spending

,id,credit
0,51043,22990
1,48329,2500
2,74943,69880
3,75029,6000
4,43018,29000


In [16]:
data = clients.merge(earnings, how='left', on='id').merge(spending, how='left', on='id')
data[data['debit'] < data['credit']]

,id,name,debit,credit
0,43018,Марков Илья,19800,29000
4,75029,Гордецкий Максим,5355,6000


In [20]:
(clients
 .merge(earnings, how='left', on='id') # объединение с расходами
 .merge(spending, how='left', on='id') # объединение с доходами
)

,id,name,debit,credit
0,43018,Марков Илья,19800,29000
1,48329,Зарицкая Елизавета,12400,2500
2,51043,Благова Дарья,34500,22990
3,74943,Слепова Елена,89044,69880
4,75029,Гордецкий Максим,5355,6000


In [18]:
cl_ear = clients.merge(earnings, how='left')
final = cl_ear.merge(spending, how='left')
final.loc[(final.debit < final.credit)].head()

,id,name,debit,credit
0,43018,Марков Илья,19800,29000
4,75029,Гордецкий Максим,5355,6000


### INNER join
Оставляет строчки, которые есть в обеих таблицах

In [21]:
visits_grouped.join(purchases_pivot, how='inner')

NameError: name 'visits_grouped' is not defined

### Outer join
Оставляет все строчки

In [ ]:
visits_grouped.join(purchases_pivot, how='outer')

# Конкатенация таблиц

In [26]:
a = pd.DataFrame({'date': ['2020-01-01', '2020-01-02', '2020-01-03'], 'value_a': [1, 2, 3]})
b = pd.DataFrame({'date': ['2020-01-01', '2020-01-02', '2020-01-03'], 'value_b': [3, 4, 5]})

In [27]:
a

,date,value_a
0,2020-01-01,1
1,2020-01-02,2
2,2020-01-03,3


In [28]:
b

,date,value_b
0,2020-01-01,3
1,2020-01-02,4
2,2020-01-03,5


In [29]:
pd.concat([a, b])

,date,value_a,value_b
0,2020-01-01,1.0,NaN
1,2020-01-02,2.0,NaN
2,2020-01-03,3.0,NaN
0,2020-01-01,NaN,3.0
1,2020-01-02,NaN,4.0
2,2020-01-03,NaN,5.0


### Дубликаты при объединении таблиц

In [30]:
ratings = pd.read_csv('ratings_example.txt', sep = '\t')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144


In [31]:
movies = pd.read_csv('movies_example.txt', sep = '\t')
movies.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
2,31,Dangerous Minds (1995),Drama


In [32]:
# ¯\_(ツ)_/¯

ratings.merge(movies, how='left', on='movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


In [33]:
movies.drop_duplicates(subset = 'movieId', keep = 'first', inplace = True)
movies.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


In [34]:
ratings.merge(movies, how = 'left', on = 'movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


In [35]:
ratings.merge(movies, how = 'right', on = 'movieId')

,userId,movieId,rating,timestamp,title,genres
0,1.0,31,2.5,1.260759e+09,Dangerous Minds (1995),Drama
1,NaN,32,NaN,NaN,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


In [ ]:
len(ratings) == len(ratings.merge(movies, how = 'right', on = 'movieId'))

### Упражнение
Объедините датафреймы с визитами и покупками на сайте по ключу date. Обратите внимание, что в датафрейме визитов имеются дубликаты по дате.

In [41]:
visits = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-01', '2019-11-02', '2019-11-02', '2019-11-03'], 
     'source': ['organic', 'paid', 'organic', 'paid', 'organic'], 
     'visits': [16825, 1952, 21890, 376, 19509]}
)

visits

,date,source,visits
0,2019-11-01,organic,16825
1,2019-11-01,paid,1952
2,2019-11-02,organic,21890
3,2019-11-02,paid,376
4,2019-11-03,organic,19509


In [42]:
orders = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-02', '2019-11-03'],
     'orders': [198, 225, 201]}
)

orders

,date,orders
0,2019-11-01,198
1,2019-11-02,225
2,2019-11-03,201


In [52]:
visits.pivot_table(index='date', columns='source', values='visits', 
                                        aggfunc='sum')

source,organic,paid
date,,
2019-11-01,16825.0,1952.0
2019-11-02,21890.0,376.0
2019-11-03,19509.0,NaN


In [38]:
visits_pivot = visits.pivot_table(index='date', columns='source', values='visits', 
                                        aggfunc='sum')

visits_pivot.merge(orders, how = 'outer', on = 'date')

,date,organic,paid,orders
0,2019-11-01,16825.0,1952.0,198
1,2019-11-02,21890.0,376.0,225
2,2019-11-03,19509.0,NaN,201


In [43]:
visits.merge(orders)

,date,source,visits,orders
0,2019-11-01,organic,16825,198
1,2019-11-01,paid,1952,198
2,2019-11-02,organic,21890,225
3,2019-11-02,paid,376,225
4,2019-11-03,organic,19509,201


In [45]:
data = visits.merge(orders, how ='outer', on = 'date')
print(data)

         date   source  visits  orders
0  2019-11-01  organic   16825     198
1  2019-11-01     paid    1952     198
2  2019-11-02  organic   21890     225
3  2019-11-02     paid     376     225
4  2019-11-03  organic   19509     201


In [46]:
data.groupby(['date', 'orders']).sum().sort_values('date')

,,visits
date,orders,
2019-11-01,198,18777
2019-11-02,225,22266
2019-11-03,201,19509


In [49]:
visit_info = visits.pivot_table(index = 'date', columns = 'source', values = 'visits', aggfunc = 'sum', 
                                fill_value = 0)
visit_info

source,organic,paid
date,,
2019-11-01,16825,1952
2019-11-02,21890,376
2019-11-03,19509,0


In [50]:
visit_info.merge(orders, how = 'outer', on = 'date')

,date,organic,paid,orders
0,2019-11-01,16825,1952,198
1,2019-11-02,21890,376,225
2,2019-11-03,19509,0,201


In [51]:
visits_gr = visits.groupby('date').sum()
full = orders.merge(visits_gr, on = 'date')
full

,date,orders,visits
0,2019-11-01,198,18777
1,2019-11-02,225,22266
2,2019-11-03,201,19509


### Оптимизация хранения данных

In [53]:
# 2.4mb
ratings = pd.read_csv('ml-latest-small/ratings.csv')

# 0.5mb
movies = pd.read_csv('ml-latest-small/movies.csv')
joined = ratings.merge(movies, how='left', on='movieId')

In [54]:
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [56]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
len(ratings)

In [57]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
len(movies)

In [ ]:
len(ratings) == len(joined)

In [55]:
joined.to_csv('joined_ratings.csv', index=False)

In [ ]:
joined.head()

In [ ]:
logs = joined[['userId', 'movieId', 'rating']].head()

In [ ]:
len(joined[['movieId', 'title', 'genres']].drop_duplicates())

### Какой жанр имеет самые высокие рейтинги?

In [58]:
import numpy as np

In [59]:
genres = ['Drama', 'Action', 'Thriller']

In [60]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [61]:
len(ratings)

100004

In [62]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [63]:
len(movies)

9125

In [64]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [ ]:
# рекомендуемая проверка на возможные дубликаты

len(ratings) == len(joined)

### Считаем рейтинг жанров

In [65]:
# еще раз список жанров

genres = ['Drama', 'Action', 'Thriller']

In [66]:
def genres_ratings(row):
    """Возвращает рейтинг, если он есть в списке жанров данного фильма"""
    
    return pd.Series([row['rating'] if genre in row['genres'] else np.NaN for genre in genres])

In [67]:
joined[genres] = joined.apply(genres_ratings, axis=1)
joined.head()

,userId,movieId,rating,timestamp,title,genres,Drama,Action,Thriller
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,2.5,NaN,NaN
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,3.0,NaN,NaN
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,NaN,NaN,3.0
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,NaN,2.0,2.0
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,4.0,NaN,NaN


In [68]:
joined.Drama.mean()

3.681779585269932

### Упражнение
Выведите средний рейтинг каждого жанра из списка genres

### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

In [ ]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

In [ ]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

In [ ]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base